# Day 4

In [29]:
directions = [
    [-1, 0],  # left (0)
    [-1, -1],  # up-left (1)
    [0, -1],  # up (2)
    [1, -1],  # up-right (3)
    [1, 0],  # right (4)
    [1, 1],  # down-right (5)
    [0, 1],  # down (6)
    [-1, 1],  # down-left (7)
]

def get_neighbor_coordinates(grid, direction, x, y):
    transformations = directions[direction]
    newX = x + transformations[0]
    newY = y + transformations[1]
    return newX, newY


def safe_get_value(grid, x, y):
    if x < 0 or y < 0 or x >= len(grid[0]) or y >= len(grid):
        return None
    return grid[y][x]


def get_neighbor(grid, direction, x, y):
    newX, newY = get_neighbor_coordinates(grid, direction, x, y)
    return safe_get_value(grid, newX, newY)


def get_matching_neighbor_directions(grid, x, y):
    matching_directions = []
    for i in range(len(directions)):
        if get_neighbor(grid, i, x, y) == "M":
            matching_directions.append(i)
    return matching_directions


def try_path(grid, x, y, direction):
    # should be M
    curr_x, curr_y = get_neighbor_coordinates(grid, direction, x, y)
    # want to find A
    curr_x, curr_y = get_neighbor_coordinates(grid, direction, curr_x, curr_y)
    cursor = safe_get_value(grid, curr_x, curr_y)
    if (cursor != "A"):
        return False
    # want to find S
    curr_x, curr_y = get_neighbor_coordinates(grid, direction, curr_x, curr_y)
    cursor = safe_get_value(grid, curr_x, curr_y)
    if (cursor != "S"):
        return False    
    return True
        

def process_puzzle(puzzle):
    rows = puzzle.split("\n")
    grid = [list(row) for row in rows]
    return grid


def runner():
    with open("./inputs/day4.txt") as f:
        day4 = f.read()
    grid = process_puzzle(day4)
    num = 0
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            cursor = grid[y][x]
            if cursor == "X":
                matching_neighbor_directions = get_matching_neighbor_directions(grid, x, y)
                for dir in matching_neighbor_directions:
                    if try_path(grid, x, y, dir) == True:
                        num += 1
    return num


def is_unique_match(corner1, corner2):
    if (corner1 != corner2) and (corner1 == "M" or corner1 == "S") and (corner2 == "M" or corner2 == "S"):
        return True
    return False


def check_diagonals(grid, x, y):
    ul = get_neighbor(grid, 1, x, y)
    dr = get_neighbor(grid, 5, x, y)
    if is_unique_match(ul, dr) == False:
        return False
    
    ur = get_neighbor(grid, 3, x, y)
    dl = get_neighbor(grid, 7, x, y)
    if is_unique_match(ur, dl) == False:
        return False
    
    return True

def runner_pt2():
    with open("./inputs/day4.txt") as f:
        day4 = f.read()
    grid = process_puzzle(day4)
    num = 0
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            cursor = grid[y][x]
            if cursor == "A":
                if check_diagonals(grid, x, y) == True:
                    num += 1
    return num

print(runner())
print(runner_pt2())

2554
1916


# Day 5

In [30]:
def read_inputs():
    with open("./inputs/day5-rules.txt") as f:
        day5_rules = f.read().splitlines()
    with open("./inputs/day5-pages.txt") as f:
        day5_pages = f.read().splitlines()
    return day5_rules, day5_pages


def process_inputs(rules, pages):
    # construct rule map where AFTER constraint is key and all BEFORE constraints are values
    # ex. {5: [1, 2, 3, 4]} indicates that 5 must come after 1, 2, 3, and 4
    rule_map = {}
    for rule in rules:
        [before, after] = [int(x) for x in rule.split("|")]
        if after in rule_map:
            rule_map[after] = rule_map[after] + [before]
        else:
            rule_map[after] = [before]
    # construct candidates as a 2D array of ints
    candidates = [[int(s) for s in x.split(",")] for x in pages]
    return rule_map, candidates


def check_candidate(rule_map, candidate):
    disallow_set = set()
    for num in candidate:
        if num in disallow_set:
            return False
        if num in rule_map:
            disallow_set.update(rule_map[num])
    return True


def fix_candidate(rule_map, candidate):
    fixed_candidate = [-1] * len(candidate)
    for num in candidate:
        if num in rule_map:
            applicable_rules = list(filter(lambda x: x in candidate, rule_map[num]))
            fixed_candidate[len(applicable_rules)] = num
    return fixed_candidate


def get_middle_value(arr):
    return arr[len(arr) // 2]


def runner():
    valid_sum = 0
    invalid_sum = 0
    rules, pages = read_inputs()
    rule_map, candidates = process_inputs(rules, pages)
    for i, candidate in enumerate(candidates):
        # part 1
        if check_candidate(rule_map, candidate):
            valid_sum += get_middle_value(candidate)
        # part 2
        else:
            fixed_candidate = fix_candidate(rule_map, candidate)
            invalid_sum += get_middle_value(fixed_candidate)
    return valid_sum, invalid_sum


runner()

(5087, 4971)

# Day 6

In [ ]:
import time
from IPython.display import clear_output, display, HTML
import html

directions = ["^", ">", "v", "<"]
directionMap = {"^": [0, -1], ">": [1, 0], "v": [0, 1], "<": [-1, 0]}


def read_inputs():
    with open("./inputs/day6.txt") as f:
        arr = f.read().splitlines()
    grid = [list(row) for row in arr]
    return grid


def rotate(direction):
    i = directions.index(direction)
    newI = (i + 1) % len(directions)
    return directions[newI]


def stringifyCoors(x, y, dir):
    return f"{x}|{y}", f"{x}|{y}|{dir}"


def pretty_print(grid):
    gridTxt = "\n".join(["".join(row) for row in grid])
    msg = html.escape(gridTxt)
    clear_output(wait=True)
    display(
        HTML(
            f"<pre style='font-family: monospace; font-size: 12px; line-height: 1'>{msg}</pre>"
        )
    )
    time.sleep(0.005)


def find_cursor(grid):
    for y, row in enumerate(grid):
        for x, item in enumerate(row):
            if item in directions:
                return [x, y]
    return None


def try_step(grid, x, y, direction):
    # pretty print for debugging and fun! comment it out for time efficiency.
    # pretty_print(grid)
    # calculate new spot
    transformation = directionMap[direction]
    newX = x + transformation[0]
    newY = y + transformation[1]

    # exited the grid
    if newX < 0 or newX >= len(grid[0]) or newY < 0 or newY >= len(grid):
        return False

    # hit a barrier
    if grid[newY][newX] == "#":
        newDir = rotate(direction)
        return x, y, newDir, False

    # perform step forward
    return newX, newY, direction, True


def compute_path(grid, startX, startY, startDir):
    cursorX, cursorY, direction = startX, startY, startDir
    coors, coors_with_dir = stringifyCoors(cursorX, cursorY, direction)
    visited = {coors}
    visited_with_dir = {coors_with_dir}
    path = [(cursorX, cursorY, direction)]

    while True:
        step = try_step(grid, cursorX, cursorY, direction)
        if step == False:
            break
        cursorX, cursorY, newDir, moved = step
        if not moved:
            direction = newDir
            continue
        direction = newDir
        coors, coors_with_dir = stringifyCoors(cursorX, cursorY, direction)
        # check for loops
        if coors_with_dir in visited_with_dir:
            return True, visited, path
        visited.add(coors)
        visited_with_dir.add(coors_with_dir)
        path.append((cursorX, cursorY, direction))
    return False, visited, path


def count_loops(grid, path):
    startX, startY, startDir = path[0]
    loops = set()
    for step in path[1:]:
        currX, currY, _ = step
        grid[currY][currX] = "#"
        isLoop, _, _ = compute_path(grid, startX, startY, startDir)
        grid[currY][currX] = "."
        if isLoop == True:
            coors, _ = stringifyCoors(currX, currY, "X")
            loops.add(coors)
    return loops


def runner():
    grid = read_inputs()
    [cursorX, cursorY] = find_cursor(grid)
    direction = grid[cursorY][cursorX]

    _, visitedSet, path = compute_path(grid, cursorX, cursorY, direction)
    loops = count_loops(grid, path)

    return len(visitedSet), len(loops)


runner()

(5531, 2165)

# Day 9

In [ ]:
def process_input():
    with open("./inputs/day9.txt") as file:
        fileStr = file.read()
    input = [int(char) for char in list(fileStr)]
    return input


def arrange(arr):
    disk = []
    file_space = []
    free_space = []
    for i, item in enumerate(arr):
        if i % 2 == 0:
            file_space.append({"index": len(disk), "space": item})
            disk.extend([i // 2] * item)
        else:
            if item > 0:
                free_space.append({"index": len(disk), "space": item})
            disk.extend(["."] * item)
    return disk, free_space, file_space


def fragment_part1(disk):
    left = 0
    right = len(disk) - 1
    target = None
    while left < right:
        if target != None:
            if disk[left] == ".":
                disk[left] = target
                disk[right] = "."
                target = None
            else:
                left += 1
            continue
        if disk[right] == ".":
            right -= 1
            continue
        else:
            target = disk[right]
            continue
    return disk


def fragment_part2(disk, free_space, file_space):
    for file in reversed(file_space):
        remove_target = None
        for index, free in enumerate(free_space):
            if free['index'] >= file['index']:
                break
            if free['space'] >= file['space']:
                target = disk[file['index']]
                for i in range(file['space']):
                    disk[free['index'] + i] = target
                    disk[file['index'] + i] = "."
                free['space'] -= file['space']
                free['index'] += file['space']
                if free['space'] <= 0:
                    remove_target = index
                break
        # This results in a significant speed up!
        if remove_target != None:
            free_space.pop(remove_target)
    return disk


def checksum(disk):
    sum = 0
    for i, item in enumerate(disk):
        if item != ".":
            sum += i * item
    return sum


def runner():
    # input = [int(char) for char in list("2333133121414131402")]
    input = process_input()
    disk, free_space, file_space = arrange(input)
    disk_two = disk.copy()

    fragment_part1(disk)
    print(checksum(disk))

    fragment_part2(disk_two, free_space, file_space)
    print(checksum(disk_two))
    
    # print("".join([str(x) for x in disk_two]))


runner()

6332189866718
6353648390778


# Day 10

In [ ]:
def process_input():
    with open("./inputs/day10.txt") as file:
        arr = file.read().splitlines()
    return [[int(c) for c in list(row)] for row in arr]


def find_trailheads(grid):
    coors = []
    for y, row in enumerate(grid):
        for x, item in enumerate(row):
            if item == 0:
                coors.append((x, y))
    return coors


def get_neighbors(grid, x, y):
    transformations = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    item = grid[y][x]
    neighbors = []
    for t in transformations:
        dx, dy = t
        newX = x + dx
        if newX < 0 or newX >= len(grid[0]):
            continue
        newY = y + dy
        if newY < 0 or newY >= len(grid):
            continue
        if item + 1 == grid[newY][newX]:
            neighbors.append((newX, newY))
    return neighbors


def dfs(grid, originX, originY, target):
    search_stack = get_neighbors(grid, originX, originY)
    paths = []
    targets = set()
    path = [(originX, originY)]
    while len(search_stack) > 0:
        neighbor = search_stack.pop()
        x, y = neighbor
        path.append(neighbor)
        if grid[y][x] == target:
            paths.append(path.copy())
            targets.add(neighbor)
            path.pop()
        else:
            new_neighbors = get_neighbors(grid, x, y)
            if len(new_neighbors) == 0:
                path.pop()
            else:
                search_stack.extend(new_neighbors)
    return paths, targets


def visualize(grid, path):
    print("Length", len(path))
    modified_grid = [[str(item) if (x, y) in path else "." for x, item in enumerate(row)] for y, row in enumerate(grid)]
    for row in modified_grid:
        print("".join(row))


def runner():
    grid = process_input()
    trailheads = find_trailheads(grid)
    part1_score = 0
    part2_rating = 0
    for t in trailheads:
        x, y = t
        paths, targets = dfs(grid, x, y, 9)
        part1_score += len(targets)
        part2_rating += len(paths)
    return part1_score, part2_rating
    

runner()

(496, 1120)